In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
# define same parameters
Class = ["No DR", "Mild", "Moderate", "Severe", "Proliferative DR"]
img_size = 224
# Define paths to images and labels
train_image_dir = (
    "/kaggle/input/aptos2019-blindness-detection/train_images"
)
train_label_file = "/kaggle/input/aptos2019-blindness-detection/train.csv"

In [ ]:
# Load labels into a pandas dataframe
labels_df = pd.read_csv(train_label_file)

In [ ]:
labels_df.head()

In [ ]:
# Create a dictionary mapping image IDs to labels
label_dict = dict(zip(labels_df["id_code"], labels_df["diagnosis"]))

In [ ]:
# A dictionary where the keys are the image IDs and the values are the diagnoses
label_dict

In [ ]:
# A function to load and preprocess images
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_size, img_size])
    image /= 255.0
    return image

In [ ]:
# A function to get the label for an image
def get_label(image_path):
    image_id = os.path.basename(image_path).split(".")[0]
    res = np.zeros(5)
    res[label_dict[image_id]] = 1
    return res

In [ ]:
get_label("/kaggle/input/aptos2019-blindness-detection/train_images/ef476be214d4.png")

In [ ]:
# list of image paths
image_paths = [
    os.path.join(train_image_dir, filename) for filename in os.listdir(train_image_dir)
]

In [ ]:
image_paths

In [ ]:
# Split the dataset into training, validation, and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths,
    [get_label(path) for path in image_paths],
    test_size=0.2,
    random_state=42,
)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.2, random_state=42
)

In [ ]:
# Create a dataset of image paths
train_paths_ds = tf.data.Dataset.from_tensor_slices(train_paths)
val_paths_ds = tf.data.Dataset.from_tensor_slices(val_paths)
test_paths_ds = tf.data.Dataset.from_tensor_slices(test_paths)

In [ ]:
# Load and preprocess the images
train_images_ds_1 = train_paths_ds.map(load_and_preprocess_image)
val_images_ds_1 = val_paths_ds.map(load_and_preprocess_image)
test_images_ds_1 = test_paths_ds.map(load_and_preprocess_image)

# Data Preprocessing 

In [ ]:
# Data Preprocessing (Circular Cropping and Gaussian Blur)
def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if check_shape == 0:  # image is too dark so that we crop out everything,
            return img  # return original image
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img


def circle_crop(img, sigmaX):
    img = img.numpy()
    img = crop_image_from_gray(img)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    height, width, depth = img.shape

    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), float(sigmaX)), -4, 0.5)
    return img

In [ ]:
# example of Performing Image Processing on an image
img = list(train_images_ds_1.take(1))[0]
img_t = circle_crop(img, sigmaX=30)

f, axarr = plt.subplots(1, 2, figsize=(11, 11))
axarr[0].imshow(img)
axarr[1].imshow(img_t)
plt.title("After applying Circular Crop and Gaussian Blur")
plt.show()

In [ ]:
# Apply the function over the dataset
train_images_ds = train_images_ds_1.map(
    lambda x: tf.py_function(circle_crop, [x, 30], tf.float32)
)
val_images_ds = val_images_ds_1.map(
    lambda x: tf.py_function(circle_crop, [x, 30], tf.float32)
)
test_images_ds = test_images_ds_1.map(
    lambda x: tf.py_function(circle_crop, [x, 30], tf.float32)
)

In [ ]:
# Combine the images and labels into a single dataset
train = tf.data.Dataset.zip(
    (train_images_ds, tf.data.Dataset.from_tensor_slices(train_labels))
)
val = tf.data.Dataset.zip(
    (val_images_ds, tf.data.Dataset.from_tensor_slices(val_labels))
)
test = tf.data.Dataset.zip(
    (test_images_ds, tf.data.Dataset.from_tensor_slices(test_labels))
)

In [ ]:
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
test = test.batch(8)
test = test.prefetch(4)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()

In [ ]:
# Assuming the train_dataset is already loaded
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()
train_plot = train.take(4)
for (img, label), ax in zip(train_plot, axs):
    ax.imshow(img.numpy()[0])
    ax.set_title(f"Label: {label.numpy()[0]}")
    ax.axis("off")

plt.show()

# Building the Model

In [ ]:
LEARNING_RATE = 1e-4
WARMUP_LEARNING_RATE = 1e-3
WARMUP_EPOCHS = 3

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
def create_model():
    # Load the pre-trained ResNet50 model
    base_model = ResNet50(
        weights="imagenet", include_top=False, input_shape=(224, 224, 3)
    )

    # Add a global average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Add a fully connected layer with 2048 neurons and ReLU activation
    x = Dense(2048, activation="relu")(x)

    # Dropout by 0.5
    x = Dropout(0.5)(x)

    # Add an output layer with 5 neurons (for the 5 classes) and softmax activation
    predictions = Dense(5, activation="softmax")(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
model = create_model()

In [ ]:
for layer in model.layers:
    layer.trainable = False

for i in range(-5, 0):
    model.layers[i].trainable = True

In [ ]:
model.summary()

In [ ]:
# see the model architecture
tf.keras.utils.plot_model(model)

#  **Training the Model**

In [ ]:
# Warm up the model to make our new layers get some initial training
model.compile(
    optimizer=optimizers.Adam(lr=WARMUP_LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

history_warmup = model.fit(train, validation_data=val, epochs=WARMUP_EPOCHS)

In [ ]:
# now make all layers trainable
for layer in model.layers:
    layer.trainable = True

In [ ]:
es = EarlyStopping(
    monitor="val_loss", mode="min", patience=5, restore_best_weights=True, verbose=1
)
rlrop = ReduceLROnPlateau(
    monitor="val_loss", mode="min", patience=3, factor=0.5, min_lr=1e-6, verbose=1
)

callback_list = [es, rlrop]
optimizer = optimizers.Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
history_finetunning = model.fit(
    train, validation_data=val, epochs=40, callbacks=callback_list, verbose=1
)

In [ ]:
plt.figure(figsize=(8, 5))

plt.plot(history_finetunning.history["accuracy"])
plt.plot(history_finetunning.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.gca().ticklabel_format(axis="both", style="plain", useOffset=False)
plt.show()

In [ ]:
model.save("CodeClauseInternship_blindnessdetection.h5")

In [ ]:
# loading the model later
from tensorflow.keras.models import load_model

model = load_model("CodeClauseInternship_blindnessdetection.h5")

In [ ]:
test_iter = test.as_numpy_iterator()
test_plot = test_iter.next()

In [ ]:
test_plot

In [ ]:
pred = model.predict(test_plot[0])

In [ ]:
pred.shape

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(10, 10))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
axs = axs.flatten()
for n, ax in enumerate(axs):
    ax.imshow(test_plot[0][n])
    ax.set_title(
        f"""Predicted Label: {Class[np.argmax(pred[n])]}
    Actual label: {Class[np.argmax(test_plot[1][n])]}"""
    )
    ax.axis("off")

plt.show()